In [18]:
import pandas as pd
from nltk.corpus import stopwords
import nltk
import re
import string
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [3]:
text_dataset = "emails.csv"

In [5]:
text_data = pd.read_csv(text_dataset)

In [6]:
text_data.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [7]:
stop_words = set(stopwords.words("english"))

In [8]:
def clean_text(text):
    text = text.lower()
    text = text[9:]
    text = re.sub("@\S+", " ", text)
    text = re.sub("https*\S+", " ", text)
    text = re.sub("#\S+", " ", text)
    text = re.sub("\d", " ", text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\s{2,}',' ', text)
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    text = text.strip()
    return text

In [10]:
for ind, text in enumerate(text_data['text']):
    text = clean_text(text)
    text_data['text'][ind] = text

C:\Users\it02102021\AppData\Local\Temp\ipykernel_2160\3449085219.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_data['text'][ind] = text


In [11]:
text_data['text']

0       irresistible corporate identity lt really hard...
1       ding gunslinger fanny merrill muzo colza attai...
2       ble new homes made easy im wanting show homeow...
3       nting special request additional information c...
4       software cds software compatibility great grow...
                              ...                        
5723    development charges gpg forwarded shirley cren...
5724    visit jim thanks invitation visit lsu shirley ...
5725    e study update wow day super thank much vince ...
5726    david please call shirley crenshaw assistant e...
5727    ra update aurora version fastest model got fas...
Name: text, Length: 5728, dtype: object

In [12]:
X = text_data['text']
y = text_data['spam']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [14]:
max_words = 10000
max_seq_len = 100

In [19]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

In [20]:
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_seq_len)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_seq_len)

In [21]:
emb_dim = 128

In [23]:
model = keras.Sequential([
    layers.Embedding(max_words, emb_dim),
    layers.LSTM(128),
    layers.Dense(1, activation='sigmoid')
])

In [24]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [29]:
model.fit(X_train, y_train, epochs=3, batch_size=128, validation_split=0.3)

Epoch 1/3
22/22 [==============================] - 3s 155ms/step - loss: 8.9381e-04 - accuracy: 0.9996 - val_loss: 0.0479 - val_accuracy: 0.9867
Epoch 2/3
22/22 [==============================] - 3s 151ms/step - loss: 8.6148e-04 - accuracy: 0.9996 - val_loss: 0.0489 - val_accuracy: 0.9850
Epoch 3/3
22/22 [==============================] - 3s 152ms/step - loss: 8.3835e-04 - accuracy: 0.9996 - val_loss: 0.0501 - val_accuracy: 0.9859


In [40]:
accuracy = model.evaluate(X_test, y_test)[1]
print(f"accuracy: {round(accuracy, 3)}")

54/54 [==============================] - 1s 15ms/step - loss: 0.0692 - accuracy: 0.9825
accuracy: 0.983
